From government data I get data in ascii-grid format which I need to transform into latitude/longitude data.
Using the data I can determine the lowest pollution rate.
This information is available on https://geodata.rivm.nl/gcn/. I will use data of 2018.

# Using cell size of 3 km x 3 km
The PM25 data has a shape of 320*280 = 89600 cells.
Each cell represents 1 km x 1 km

89600 is too much to handle using Nominatim to get address details.
Nominatim limits the number of calls to max 1 call per second.
I set the sleep timer to 2 seconds to be sure I am within limits.

I will change the cell size to 3 km x 3 km which still gives good results for PM25 data.
Thereby reducing the number of cells to 4886.

In [2]:
# Generic imports

In [1]:
import pandas as pd
import time as time
import numpy as np

# Retrieve Pollution Rate Data from Government site
The dataset is downloaded from https://geodata.rivm.nl/gcn/ in a ASCII-GRID format. 
See also https://en.wikipedia.org/wiki/Esri_grid

Location in grid is according to the 'Amersfoortse coordinaten' which is described in dutch on
https://nl.wikipedia.org/wiki/Rijksdriehoeksco%C3%B6rdinaten
Amersfoort has a location of X=155000 and Y=463000

First 6 lines show
* NCOLS          280  (nr of cells in X direction)
* NROWS          320  (nr of cells in Y direction)
* XLLCORNER        0  (X coordinate)
* YLLCORNER       300000  Y Coordinate
* CELLSIZE         1000  (1000 meters so 1 km)
* NODATA_VALUE  -0.9990E+03 (this is the value if there is no data)


To convert the data into longitude and latitude I use code from https://thomasv.nl/2014/03/rd-naar-gps/
This converts the so called Rijksdriehoek coordinates into latitude and longitude. 
Code is a separate class called RDWGSConverter


In [2]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="test1")
from RDWGSConverter import RDWGSConverter  # import module to convert Rijksdriehoek coordinates into latitude and longitude


# Load Pollution Data into a Dataframe

In [3]:
listOfColumnWidth = [13] * 280  # 280 columns of width 13
df_pm25 = pd.read_fwf('conc_pm25_2018/conc_pm25_2018.asc', widths=listOfColumnWidth,skiprows=6,header=None)

#### Replace missing values in the data(which is represented as -999.0) by nan

In [4]:
df_pm25.replace(-999.0, np.nan,inplace=True)
df_pm25.describe()

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
count,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,4.000000,6.000000,9.000000,...,54.000000,53.000000,51.000000,49.000000,47.000000,46.000000,32.000000,13.000000,1.000,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,9.634000,9.626750,9.614167,9.595444,...,8.407759,8.416208,8.476588,8.506490,8.513638,8.490870,8.541688,8.576846,8.418,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,0.007071,0.009777,0.012734,0.014028,...,0.331099,0.336813,0.341648,0.379752,0.380547,0.371899,0.336820,0.241870,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,9.629000,9.616000,9.596000,9.573000,...,7.751000,7.762000,7.785000,7.801000,7.820000,7.843000,7.879000,8.252000,8.418,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,9.631500,9.619750,9.605500,9.584000,...,8.322000,8.222000,8.280500,8.280000,8.281000,8.269750,8.352000,8.316000,8.418,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,9.634000,9.627500,9.616500,9.598000,...,8.459000,8.498000,8.617000,8.602000,8.568000,8.496500,8.629000,8.723000,8.418,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,9.636500,9.634500,9.624500,9.608000,...,8.650250,8.651000,8.723000,8.801000,8.820500,8.834750,8.824000,8.783000,8.418,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,9.639000,9.636000,9.627000,9.611000,...,8.917000,8.866000,8.855000,9.016000,8.995000,8.982000,8.928000,8.836000,8.418,NaN


# Convert grid data into a single column and determine x and y values

The matrix of 320*280 needs to be converted to a list with 3 columns
latitude, longitude and the value of the relevant cell in the matrix
So I need to read in a loop 320*280 and based on that I can determine the X and Y value in Rijkscoordinaten
and then convert it into latitude,longitude.
Only keep cells which do not contain NAN.

Also reduce the number of cells used by using a cell size of 3 km x 3 km.
This will be achieved by stepping through the grid with a step of 3.



In [16]:

conv = RDWGSConverter()

area_size=3 # set to 3 km. A cell in the loaded grid is (1 KM)

# netherlands is 320km high by 280 wide
# nr of rows is 320. so it loops over rows which is y range !!!
# A row number starts with 0 and the increments
# The Y coordinate starts at 620 and decrements.
# So row 0 is y-coordinate 620
# y_top is used to determine correct position in the grid
# Amsterdam is on rijkscoordinaat 487432 in Y direction
# This needs to convert to 487432/1000 = apx 487
# then position in the grid is on 620-487=133
# Coordinates are represented in meters. So I have to multiply a grid position by 1000 meters since a cell is 1 km.
y_top = 620  # this is used to calculate correct position in the grid
df_limit = df_pm25

x_range=range(0,df_limit.shape[1],area_size)   
y_range=range(0,df_limit.shape[0],area_size) # loop from highest point to lowest point
df1 = pd.DataFrame()
k = 0
for rd_y in y_range:
    for rd_x in x_range:
        if pd.notnull(df_limit.iat[rd_y,rd_x]): 
            #print(i,j,df_limit.iat[i,j])
            df1.loc[k,'pm25'] = df_limit.iat[rd_y,rd_x]
            
            wgsCoords = conv.fromRdToWgs( [rd_x*1000,(y_top-rd_y)*1000] )
            #print(i,j,k,wgsCoords)
            df1.loc[k,'lat'] =  wgsCoords[0]
            df1.loc[k,'long'] = wgsCoords[1]
            df1.loc[k,'rd_x'] = rd_x*1000
            df1.loc[k,'rd_y'] = (y_top-rd_y)*1000
            k = k + 1
df1.shape            

(4886, 5)

In [17]:
df1.head()


,pm25,lat,long,rd_x,rd_y
0,7.675,53.563184,6.217233,210000.0,620000.0
1,7.672,53.562865,6.262502,213000.0,620000.0
2,7.669,53.562528,6.307770,216000.0,620000.0
3,7.666,53.562174,6.353038,219000.0,620000.0
4,7.671,53.561803,6.398304,222000.0,620000.0


In [18]:
# A city is not always returned. sometimes it is village, town etc.
# I will only use data from real cities
# A valid location can be a city, town or village
# display_name is always there (combiningy country,city,village etc)
# So for this assignment diplay_name is ok.

for i in range(df1.shape[0]):
    #print(i)
    #wgsCoords = conv.fromRdToWgs( [110000,i] )
    wgsCoords = [df1.iat[i,1],df1.iat[i,2]]
    #print(wgsCoords)
    location = geolocator.reverse(wgsCoords,timeout=60) 
    # I use for location the display_name
    # places can be suburb,town,city,village and hamlets. Not all these identiers are always in JSON
    # display_name is always there
    try:
        #df1.loc[i,'suburb'] = location.raw['address']['suburb']
        df1.loc[i,'location'] = location.raw['display_name']
        print(i,'location:',location.raw['display_name'])
        #print(i,'city:',location.raw['address']['city'],',',location.raw['address']['suburb'])
        #df1.loc[i,'city'] = location.raw['address']['city']
    except:
        print(location.raw)     
        pass
    time.sleep(2) # set sleep to 2 seconds to prevent error: 429 too may requests


0 location: Nederland
1 location: Nederland
2 location: Nederland
3 location: Nederland
4 location: Nederland
5 location: Warffum, Het Hogeland, Groningen, Nederland
6 location: Warffum, Het Hogeland, Groningen, Nederland
7 location: Warffum, Het Hogeland, Groningen, Nederland
8 location: Warffum, Het Hogeland, Groningen, Nederland
9 location: Uithuizen, Leer (Ostfriesland), Niedersachsen, Deutschland
10 location: Uithuizen, Leer (Ostfriesland), Niedersachsen, Deutschland
11 location: Nederland
12 location: Nederland
13 location: Nederland
14 location: Nederland
15 location: Nederland
16 location: Nederland
17 location: Nederland
18 location: Nederland
19 location: Nederland
20 location: Warffum, Het Hogeland, Groningen, Nederland
21 location: Warffum, Het Hogeland, Groningen, Nederland
22 location: Warffum, Het Hogeland, Groningen, Nederland
23 location: Warffum, Het Hogeland, Groningen, Nederland
24 location: Warffum, Het Hogeland, Groningen, Nederland
25 location: Warffum, Het Hogel

164 location: Westernieland, Het Hogeland, Groningen, Nederland, 9956TC, Nederland
165 location: Usquert, Het Hogeland, Groningen, Nederland, 9988TD, Nederland
166 location: Middendijk, Usquert, Het Hogeland, Groningen, Nederland, 9988TC, Nederland
167 location: 5, Lauwersdwarsweg, Het Hogeland, Groningen, Nederland, 9981TP, Nederland
168 location: 43, Hefswalsterweg, Uithuizermeeden, Het Hogeland, Groningen, Nederland, 9982TP, Nederland
169 location: 5, Hooilandseweg, Uithuizermeeden, Het Hogeland, Groningen, Nederland, 9982CA, Nederland
170 location: Toppinga's-weg, Oudeschip, Het Hogeland, Groningen, Nederland, 9984XE, Nederland
171 location: Dijkweg, Spijk, Delfzijl, Groningen, Nederland, 9984NT, Nederland
172 location: Eemshaven, Het Hogeland, Groningen, Nederland
173 location: Delfzijl, Groningen, Nederland
174 location: Nederland
175 location: Nederland
176 location: Nederland
177 location: West-Terschelling, Terschelling, Fryslân, Nederland
178 location: West-Terschelling, Ters

280 location: Blije, Noardeast-Fryslân, Fryslân, Nederland, 9172AR, Nederland
281 location: Vogelzangsterweg, Blije, Noardeast-Fryslân, Fryslân, Nederland, 9171MH, Nederland
282 location: 7, Mieddyk, Holwerd, Noardeast-Fryslân, Fryslân, Nederland, 9171GD, Nederland
283 location: Heemstrawei, Foudgum, Noardeast-Fryslân, Fryslân, Nederland, 9154AW, Nederland
284 location: Dokkumerwei, Hantumeruitburen, Noardeast-Fryslân, Fryslân, Nederland, 9146CD, Nederland
285 location: Dokkumerwei, Wetsens, Noardeast-Fryslân, Fryslân, Nederland, 9122, Nederland
286 location: Humaldawei, Jouswier, Noardeast-Fryslân, Fryslân, Nederland, 9124ER, Nederland
287 location: 3, Hoitensreed, Ee, Noardeast-Fryslân, Fryslân, Nederland, 9131EK, Nederland
288 location: Jaap Poesgat, Esumakeech, Engwierum, Noardeast-Fryslân, Fryslân, Nederland, 9132EM, Nederland
289 location: Warfstermolen, Noardeast-Fryslân, Fryslân, Nederland
290 location: Hooge Zuidwal, Munnekezijl, Noardeast-Fryslân, Fryslân, Nederland, 9853TJ, 

386 location: Mearikkerpaad, Wâlterswâld, Dantumadiel, Fryslân, Nederland, 9113PD, Nederland
387 location: 1C, Weardebuorsterwei, Westergeest, Noardeast-Fryslân, Fryslân, Nederland, 9295LC, Nederland
388 location: 46, De Wygeast, Oudwoude, Noardeast-Fryslân, Fryslân, Nederland, 9294KR, Nederland
389 location: Pompster Ryd, Groeneweg, Kollumerpomp, Noardeast-Fryslân, Fryslân, Nederland, 9293MC, Nederland
390 location: De Gruuts, Foijingaweg, Warfstermolen, Noardeast-Fryslân, Fryslân, Nederland, 9293LR, Nederland
391 location: 71, Olde Borchweg, Munnekezijl, Noardeast-Fryslân, Fryslân, Nederland, 9853TB, Nederland
392 location: 9, Ooster Waarddijk, Kommerzijl, Westerkwartier, Groningen, Nederland, 9881TB, Nederland
393 location: Rikkerdaweg, Niehove, Westerkwartier, Groningen, Nederland, 9884TA, Nederland
394 location: 1, Noordhornerweg, Oldehove, Westerkwartier, Groningen, Nederland, 9883TJ, Nederland
395 location: 8, Oldijk, Ezinge, Westerkwartier, Groningen, Nederland, 9891TB, Nederla

491 location: Nijlânsdyk, Koarnjum, Leeuwarden, Fryslân, Nederland, 9056LM, Nederland
492 location: Aldlânsdyk, Jelsum, Leeuwarden, Fryslân, Nederland, 9056LK, Nederland
493 location: Canterlandseweg, Miedum, Leeuwarden, Fryslân, Nederland, 9082AW, Nederland
494 location: 22, Master Sannesstrjitte, Gytsjerk, Tytsjerksteradiel, Fryslân, Nederland, 9061CH, Nederland
495 location: Ottemaweg, Mûnein, Tytsjerksteradiel, Fryslân, Nederland, 9254HG, Nederland
496 location: 1-003, Johanneswâld, Feanwâlden, Dantumadiel, Fryslân, Nederland, 9269VS, Nederland
497 location: Kûkhernsterpaed, Twijzelerheide, Achtkarspelen, Fryslân, Nederland, 9287, Nederland
498 location: Mounewei, Twijzelerheide, Achtkarspelen, Fryslân, Nederland, 9287VL, Nederland
499 location: Simkepaad, Twijzel, Achtkarspelen, Fryslân, Nederland, 9286GC, Nederland
500 location: Lutkepost, Buitenpost, Achtkarspelen, Fryslân, Nederland, 9285XE, Nederland
501 location: 2A, Molenstraat, Gerkesklooster, Achtkarspelen, Fryslân, Nederl

...

4465 location: 20, Bong, Velden, Venlo, Limburg, Nederland, 5941NL, Nederland
4466 location: Nederland - Belgique / België / Belgien
4467 location: Vlissingen, Zeeland, Nederland
4468 location: DOK 14, Zwartepolderweg, Cadzand, Sluis, Zeeland, Nederland, 4506HV, Nederland
4469 location: Zeedijk, Perenhof, Nieuwvliet, Sluis, Zeeland, Nederland, 4504AC, Nederland
4470 location: 6, Puijendijk, Groede, Sluis, Zeeland, Nederland, 4503PL, Nederland
4471 location: N676, Breskens, Sluis, Zeeland, Nederland
4472 location: Breskens, Sluis, Zeeland, Nederland
4473 location: Hoofdplaat, Sluis, Zeeland, Nederland
4474 location: Hoofdplaat, Sluis, Zeeland, Nederland
4475 location: Ritthem, Vlissingen, Zeeland, Nederland
4476 location: Borssele, Borsele, Zeeland, Nederland
4477 location: Trenteweg, Ellewoutsdijk, Borsele, Zeeland, Nederland, 4437NA, Nederland
4478 location: Iseweg, Oudelande, Borsele, Zeeland, Nederland, 4436NC, Nederland
4479 location: Langepolderweg, Baarland, Borsele, Zeeland, Ned

4563 location: Moesweg, Helenaveen, Deurne, Noord-Brabant, Nederland, 5759PR, Nederland
4564 location: De Peelse Golf, Maasduinenweg, Evertsoord, Horst aan de Maas, Limburg, Nederland, 5977NP, Nederland
4565 location: Bultenbroek, Breetse Peelweg, Sevenum, Peel en Maas, Limburg, Nederland, 5993RZ, Nederland
4566 location: 6, Rozendaal, Achter de Hoven, Maasbree, Peel en Maas, Limburg, Nederland, 5993PA, Nederland
4567 location: Crayelheide, Heibeemdenweg, Venlo, Limburg, Nederland, 5927PJ, Nederland
4568 location: 41, Aan de Wassum, Smeliënkamp, Venlo, Limburg, Nederland, 5925HA, Nederland
4569 location: 8012, Beethovenstraat, Venlo, Limburg, Nederland, 5914BN, Nederland
4570 location: 87, Niederdorfer Straße, Straelen, Kreis Kleve, Regierungsbezirk Düsseldorf, Nordrhein-Westfalen, 47638, Deutschland
4571 location: Zwinstraat, Knokke, Knokke-Heist, Brugge, West-Vlaanderen, Vlaanderen, 8300, België - Belgique - Belgien
4572 location: Terhofstededijk, Zuidzande, Sluis, Zeeland, Nederland

4653 location: Tolbergweg, Ospel, Nederweert, Limburg, Nederland, 6035RM, Nederland
4654 location: 23, Donkerpeelkensweg, Meijel, Peel en Maas, Limburg, Nederland, 5768PS, Nederland
4655 location: Snep, Noordervaart, Beringe, Peel en Maas, Limburg, Nederland, 5986AC, Nederland
4656 location: 34, Zelen, Panningen, Peel en Maas, Limburg, Nederland, 5981PN, Nederland
4657 location: Klaver, Helden, Peel en Maas, Limburg, Nederland, 5988JD, Nederland
4658 location: Boekenderbos, De Meeren, Baarlo, Peel en Maas, Limburg, Nederland, 5988NP, Nederland
4659 location: 45d, Helling, Baarlo, Peel en Maas, Limburg, Nederland, 5991BD, Nederland
4660 location: Sportpark de Bakenbos, Bakenbosweg, Tegelen, Venlo, Limburg, Nederland, 5932AD, Nederland
4661 location: Pietersdijk, Sluis, Zeeland, Nederland, 4524KR, Nederland
4662 location: Bewester Eede, Aardenburg, Sluis, Zeeland, Nederland, 4527AJ, Nederland
4663 location: Boomdijk, Aardenburg, Sluis, Zeeland, Nederland, 4527EG, Nederland
4664 location:

4745 location: 2, Wienerte, Neer, Leudal, Limburg, Nederland, 6086NG, Nederland
4746 location: Schoolbroek, A73, Beesel, Limburg, Nederland, 6071LJ, Nederland
4747 location: RWZI Zelzate, Karnemelkstraat, Wittouck, Zelzate, Eeklo, Oost-Vlaanderen, Vlaanderen, 9060, België / Belgique / Belgien
4748 location: 4, Kromhoekseweg, Overslag, Terneuzen, Zeeland, Nederland, 4575ND, Nederland
4749 location: De Hoort, Hoortweg, Budel-Dorplein, Cranendonck, Noord-Brabant, Nederland, 6024RD, Nederland
4750 location: Laurabossen, Lozerweg, Weert, Limburg, Nederland, 6002SX, Nederland
4751 location: 5, Heihuisweg, Weert, Limburg, Nederland, 6006ST, Nederland
4752 location: Dijkerpeelweg, Weert, Limburg, Nederland, 6006TX, Nederland
4753 location: Laagheideweg, Weert, Limburg, Nederland, 6005PV, Nederland
4754 location: 16, Molenveldweg, Ell, Leudal, Limburg, Nederland, 6011ST, Nederland
4755 location: Heideweg, Kelpen-Oler, Leudal, Limburg, Nederland, 6037NW, Nederland
4756 location: Abenhofweg, Baex

4830 location: 7, Schuinstraat, Maria Gewanden, Hoensbroek, Heerlen, Limburg, Nederland, 6432CM, Nederland
4831 location: Hitjesweg, Nieuw-Einde, Heerlen, Limburg, Nederland, 6413TV, Nederland
4832 location: Brunssummerheide, Kamperheideweg, Heerlen, Limburg, Nederland, 6373AB, Nederland
4833 location: 12, Merckelbaghstraat, Abdissenbosch, Landgraaf, Limburg, Nederland, 6374AX, Nederland
4834 location: Valkerhofstadt, Palenberg, Übach-Palenberg, Kreis Heinsberg, Regierungsbezirk Köln, Nordrhein-Westfalen, 52531, Deutschland
4835 location: Kruisweg, Maastricht, Limburg, Nederland, 6223HG, Nederland
4836 location: Golf Club Meerssen, Hoolhuis, Bunde, Meerssen, Limburg, Nederland, 6241GP, Nederland
4837 location: Waterval, Genzon, Ulestraten, Meerssen, Limburg, Nederland, 6235NB, Nederland
4838 location: Groot-Haasdal, Schimmert, Beekdaelen, Limburg, Nederland, 6333AX, Nederland
4839 location: Kersboomkensweg, Hulsberg, Beekdaelen, Limburg, Nederland, 6363CC, Nederland
4840 location: Jan 

http://www.nationaalgeoregister.nl/geonetwork/srv/dut/catalog.search#/metadata/29c17585-e702-463f-a5dc-99d34b17d333


In [131]:
df_pm25.shape

(320, 280)

In [198]:
df_pm25.loc[117:121,163:170]

,163,164,165,166,167,168,169,170
117,9.813,9.602,9.565,9.575,9.720,9.620,9.630,9.752
118,9.749,9.642,9.568,9.945,10.050,9.700,9.749,9.634
119,9.832,9.587,9.445,9.629,9.648,9.662,9.661,9.558
120,9.708,9.444,9.459,9.651,9.594,9.694,9.715,9.686
121,9.659,9.643,9.744,9.751,9.654,9.748,9.907,9.832


In [19]:
df1.tail(30)

,pm25,lat,long,rd_x,rd_y,location
4856,12.45,50.842425,5.699533,177000.0,317000.0,"John F. Kennedybrug, Villapark, Maastricht, Li..."
4857,11.45,50.842302,5.742123,180000.0,317000.0,"Akersteenweg, Cadier en Keer, Maastricht, Limb..."
4858,11.28,50.842163,5.784713,183000.0,317000.0,"Rootherweg, Bemelen, Eijsden-Margraten, Limbur..."
4859,11.53,50.842009,5.827302,186000.0,317000.0,"13A, Sibberkerkstraat, Valkenburg, Valkenburg ..."
4860,11.38,50.841839,5.869891,189000.0,317000.0,"Engwegen, Schin op Geul, Valkenburg aan de Geu..."
4861,11.44,50.841653,5.912479,192000.0,317000.0,"Oude Elkenraderweg, Elkenrade, Gulpen-Wittem, ..."
4862,11.69,50.841452,5.955068,195000.0,317000.0,"Trintelerbosweg, Eys, Simpelveld, Limburg, Ned..."
4863,11.70,50.841235,5.997655,198000.0,317000.0,"A76, Heerlen, Limburg, Nederland, 6422PT, Nede..."
4864,11.65,50.815459,5.699351,177000.0,314000.0,"De Maasplassen Naaktstrand, Hoge Kanaaldijk, S..."
4865,11.39,50.815336,5.741916,180000.0,314000.0,"Op de Rieseberg, Cadier en Keer, Eijsden-Margr..."


In [20]:
df1.describe()

,pm25,lat,long,rd_x,rd_y
count,4886.000000,4886.000000,4886.000000,4886.000000,4886.000000
mean,10.096868,52.320685,5.488309,161489.766680,481794.924273
std,1.493127,0.670463,0.886914,60598.848739,74563.736297
min,7.340000,50.760340,3.237452,6000.000000,308000.000000
25%,8.641250,51.758297,4.878208,120000.000000,419000.000000
50%,10.120000,52.297596,5.575333,168000.000000,479000.000000
75%,11.460000,52.892122,6.142847,207000.000000,545000.000000
max,14.380000,53.563184,7.201489,276000.000000,620000.000000


In [21]:
df1.to_csv(r'df1-Netherlands-3-by-3-km.csv')

In [222]:
df_pm25.to_csv(r'df_pm25.csv')